In [ ]:
import sys

sys.path.append("~/RMNIM")
from ip.graph_nx import Graph  # type: ignore

In [ ]:
import numpy as np
from scipy import ndimage as ndi
from skimage.util import img_as_bool
from skimage.morphology import skeletonize

In [ ]:
from image_io import *
from utils import *
from cevd import *
from visualize import *
from vfc import *
from dst_fields import *

In [ ]:
# read dataset
OP_ = [
    load_3d_volume("data/OP_1"),
    load_3d_volume("data/OP_2"),
    load_3d_volume("data/OP_3"),
    load_3d_volume("data/OP_4"),
    load_3d_volume("data/OP_5"),
    load_3d_volume("data/OP_6"),
    load_3d_volume("data/OP_7"),
    load_3d_volume("data/OP_8"),
    load_3d_volume("data/OP_9"),
]

OP_Gf_ = [
    ndi.minimum_filter(ndi.gaussian_filter(OP_[0], 2.0), 2),
    ndi.minimum_filter(ndi.gaussian_filter(OP_[1], 2.0), 2),
    ndi.minimum_filter(ndi.gaussian_filter(OP_[2], 2.0), 2),
    ndi.minimum_filter(ndi.gaussian_filter(OP_[3], 2.0), 2),
    ndi.minimum_filter(ndi.gaussian_filter(OP_[4], 2.0), 2),
    ndi.minimum_filter(ndi.gaussian_filter(OP_[5], 2.0), 2),
    ndi.minimum_filter(ndi.gaussian_filter(OP_[6], 2.0), 2),
    ndi.minimum_filter(ndi.gaussian_filter(OP_[7], 2.0), 2),
    ndi.minimum_filter(ndi.gaussian_filter(OP_[8], 2.0), 2),
]

root_ = [
    (0, 429, 31),
    (25, 391, 1),
    (38, 179, 94),
    (0, 504, 128),
    (33, 264, 185),
    (10, 412, 15),
    (39, 216, 120),
    (55, 181, 119),
    (4, 364, 64),
]

In [ ]:
# coarse segmentation
OP_[0][OP_Gf_[0] <= 0] = 0
OP_[1][OP_Gf_[1] <= 0] = 0
OP_[2][OP_Gf_[2] <= 0] = 0
OP_[3][OP_Gf_[3] <= 0] = 0
OP_[4][OP_Gf_[4] <= 0] = 0
OP_[5][OP_Gf_[5] <= 0] = 0
OP_[6][OP_Gf_[6] <= 0] = 0
OP_[7][OP_Gf_[7] <= 0] = 0
OP_[8][OP_Gf_[8] <= 0] = 0

In [ ]:
# strels
strel1 = ndi.generate_binary_structure(3, 1)
strel2 = ndi.generate_binary_structure(3, 3)

In [ ]:
distances_f1 = DistanceFields(
    volume=OP_[0],
    sigma_range=(1, 7, 2),
    neuron_threshold=0.05,
    seed_point=root_[0],
)

img_filtered = distances_f1.multiscale_anisotropic()

img_mask = distances_f1.adaptive_mean_mask(img_filtered)[0]

clean_img_mask = img_as_bool(distances_f1.morphological_denoising(img_mask))

seg_vol = distances_f1.volume_segmentation(clean_img_mask, OP_[0])

pressure_field = distances_f1.pressure_field(clean_img_mask)

thrust_field = distances_f1.thrust_field(clean_img_mask)

In [ ]:
pressure_f_gaussian = ndi.gaussian_filter(pressure_field, 1.0)
thrust_f_gaussian = ndi.gaussian_filter(thrust_field, 0.5)

In [ ]:
maximas_set = distances_f1.find_thrust_maxima(thrust_f_gaussian, clean_img_mask, order=3)
skel_coords = distances_f1.generate_skel_from_seed(
    maximas_set, root_[0], pressure_f_gaussian, clean_img_mask
)

In [ ]:
skel = create_maxima_image(skel_coords, OP_[0].shape)

In [ ]:
# distances_f1.set_skeleton(skel)
# skel_OP1 = distances_f1.get_skeleton()

In [ ]:
skeleton = skeletonize(skel)

In [ ]:
# 5. Geração do Grafo
g = Graph(skeleton)
g.create_graph(moving_avg=True)

g.prune_by_branch_length(10)

# 7. Validação da Raiz
pruned_skel_img = np.zeros(OP_[0].shape, dtype=np.uint8)
if g.get_graph().number_of_nodes() > 0:
    nodes = np.array(list(g.get_graph().nodes())).astype(int)
    pruned_skel_img[nodes[:, 0], nodes[:, 1], nodes[:, 2]] = 255

In [ ]:
g.get_graph().number_of_nodes()

In [ ]:
g.set_root(distances_f1.correct_and_update_root(pruned_skel_img, root_[0]))  # Use a raiz validada!
labeled_mst = g.apply_dfs_and_label_nodes()
output_filename = "neuron_1.5_reconstruction_robust.swc"
success = g.save_to_swc(labeled_mst, output_filename, pressure_f_gaussian)

In [ ]:
plot_projections(
    images=[
        img_filtered,
        seg_vol,
        img_mask,
        thrust_field,
        thrust_f_gaussian,
        pressure_field,
        pressure_f_gaussian,
        skel,
        skeleton,
        pruned_skel_img,
    ],
    aggregations="max",
    axes=0,
    cmaps="viridis",
    title="OP_1 ",
)

In [ ]:
distances_f2 = DistanceFields(
    volume=OP_[1],
    sigma_range=(1, 7, 2),
    neuron_threshold=0.05,
    seed_point=root_[1],
)

img_filtered = distances_f2.multiscale_anisotropic()

img_mask = distances_f2.adaptive_mean_mask(img_filtered)[0]

clean_img_mask = img_as_bool(distances_f2.morphological_denoising(img_mask))

seg_vol = distances_f2.volume_segmentation(clean_img_mask, OP_[1])

pressure_field = distances_f2.pressure_field(clean_img_mask)

thrust_field = distances_f2.thrust_field(clean_img_mask)

# skel = skeletonize(clean_img_mask)

In [ ]:
pressure_f_gaussian = ndi.gaussian_filter(pressure_field, 1.0)
thrust_f_gaussian = ndi.gaussian_filter(thrust_field, 0.5)

In [ ]:
maximas_set = distances_f2.find_thrust_maxima(thrust_f_gaussian, clean_img_mask, order=3)
skel_coords = distances_f2.generate_skel_from_seed(
    maximas_set, root_[1], pressure_f_gaussian, clean_img_mask
)

In [ ]:
skel = create_maxima_image(skel_coords, OP_[1].shape)

In [ ]:
# distances_f1.set_skeleton(skel)
# skel_OP1 = distances_f1.get_skeleton()

In [ ]:
skeleton = skeletonize(skel)

In [ ]:
g = Graph(skeleton)
g.create_graph(moving_avg=True)

g.prune_by_branch_length(10)

pruned_skel_img = np.zeros(OP_[1].shape, dtype=np.uint8)
if g.get_graph().number_of_nodes() > 0:
    nodes = np.array(list(g.get_graph().nodes())).astype(int)
    pruned_skel_img[nodes[:, 0], nodes[:, 1], nodes[:, 2]] = 255

In [ ]:
g.get_graph().number_of_nodes()

In [ ]:
g.set_root(distances_f1.correct_and_update_root(pruned_skel_img, root_[1]))  # Use a raiz validada!
labeled_mst = g.apply_dfs_and_label_nodes()
output_filename = "neuron_2.1_reconstruction_robust.swc"
success = g.save_to_swc(labeled_mst, output_filename, pressure_f_gaussian)

In [ ]:
plot_projections(
    images=[
        img_filtered,
        seg_vol,
        img_mask,
        thrust_field,
        thrust_f_gaussian,
        pressure_field,
        pressure_f_gaussian,
        skel,
        skeleton,
        pruned_skel_img,
    ],
    aggregations="max",
    axes=0,
    cmaps="viridis",
    title="OP_2 ",
)

In [ ]:
distances_f3 = DistanceFields(
    volume=OP_[2],
    sigma_range=(1, 7, 2),
    neuron_threshold=0.05,
    seed_point=root_[2],
)

img_filtered = distances_f3.multiscale_anisotropic()

img_mask = distances_f3.adaptive_mean_mask(img_filtered)[0]

clean_img_mask = img_as_bool(distances_f3.morphological_denoising(img_mask))

seg_vol = distances_f3.volume_segmentation(clean_img_mask, OP_[2])

pressure_field = distances_f3.pressure_field(clean_img_mask)

thrust_field = distances_f3.thrust_field(clean_img_mask)

In [ ]:
pressure_f_gaussian = ndi.gaussian_filter(pressure_field, 1.0)
thrust_f_gaussian = ndi.gaussian_filter(thrust_field, 0.5)

In [ ]:
maximas_set = distances_f3.find_thrust_maxima(thrust_f_gaussian, clean_img_mask, order=3)
skel_coords = distances_f3.generate_skel_from_seed(
    maximas_set, root_[2], pressure_f_gaussian, clean_img_mask
)

In [ ]:
skel = create_maxima_image(skel_coords, OP_[2].shape)

In [ ]:
skeleton = skeletonize(skel)

In [ ]:
# 5. Geração do Grafo
g = Graph(skeleton)
g.create_graph(moving_avg=True)

g.prune_by_branch_length(10)

# 7. Validação da Raiz
pruned_skel_img = np.zeros(OP_[2].shape, dtype=np.uint8)
if g.get_graph().number_of_nodes() > 0:
    nodes = np.array(list(g.get_graph().nodes())).astype(int)
    pruned_skel_img[nodes[:, 0], nodes[:, 1], nodes[:, 2]] = 255

In [ ]:
g.get_graph().number_of_nodes()

In [ ]:
g.set_root(distances_f3.correct_and_update_root(pruned_skel_img, root_[2]))  # Use a raiz validada!
labeled_mst = g.apply_dfs_and_label_nodes()
output_filename = "neuron_3.0_reconstruction_robust.swc"
success = g.save_to_swc(labeled_mst, output_filename, pressure_f_gaussian)

In [ ]:
plot_projections(
    images=[
        img_filtered,
        seg_vol,
        img_mask,
        thrust_field,
        thrust_f_gaussian,
        pressure_field,
        pressure_f_gaussian,
        skel,
        skeleton,
        pruned_skel_img,
    ],
    aggregations="max",
    axes=0,
    cmaps="viridis",
    title="OP_1 ",
)

In [ ]:
distances_f4 = DistanceFields(
    volume=OP_[3],
    sigma_range=(1, 7, 2),
    neuron_threshold=0.05,
    seed_point=root_[3],
)

img_filtered = distances_f4.multiscale_anisotropic()

img_mask = distances_f4.adaptive_mean_mask(img_filtered)[0]

clean_img_mask = img_as_bool(distances_f4.morphological_denoising(img_mask))

seg_vol = distances_f4.volume_segmentation(clean_img_mask, OP_[3])

pressure_field = distances_f4.pressure_field(clean_img_mask)

thrust_field = distances_f4.thrust_field(clean_img_mask)

In [ ]:
pressure_f_gaussian = ndi.gaussian_filter(pressure_field, 1.0)
thrust_f_gaussian = ndi.gaussian_filter(thrust_field, 0.5)

In [ ]:
maximas_set = distances_f4.find_thrust_maxima(thrust_f_gaussian, clean_img_mask, order=3)
skel_coords = distances_f4.generate_skel_from_seed(
    maximas_set, root_[3], pressure_f_gaussian, clean_img_mask
)

In [ ]:
skel = create_maxima_image(skel_coords, OP_[3].shape)

In [ ]:
skeleton = skeletonize(skel)

In [ ]:
# 5. Geração do Grafo
g = Graph(skeleton)
g.create_graph(moving_avg=True)

g.prune_by_branch_length(10)

# 7. Validação da Raiz
pruned_skel_img = np.zeros(OP_[3].shape, dtype=np.uint8)
if g.get_graph().number_of_nodes() > 0:
    nodes = np.array(list(g.get_graph().nodes())).astype(int)
    pruned_skel_img[nodes[:, 0], nodes[:, 1], nodes[:, 2]] = 255

In [ ]:
g.get_graph().number_of_nodes()

In [ ]:
g.set_root(distances_f3.correct_and_update_root(pruned_skel_img, root_[3]))  # Use a raiz validada!
labeled_mst = g.apply_dfs_and_label_nodes()
output_filename = "neuron_4.0_reconstruction_robust.swc"
success = g.save_to_swc(labeled_mst, output_filename, pressure_f_gaussian)

In [ ]:
plot_projections(
    images=[
        img_filtered,
        seg_vol,
        img_mask,
        thrust_field,
        thrust_f_gaussian,
        pressure_field,
        pressure_f_gaussian,
        skel,
        skeleton,
        pruned_skel_img,
    ],
    aggregations="max",
    axes=0,
    cmaps="viridis",
    title="OP_4 ",
)